# US Covid-19 Cases and Deaths Visualization
## County Level Static Map and State Level Dynamic Map

This project is an offshoot of a Medium article by Terence Shin titled "How to Visualize the Coronavirus Pandemic with Choropleth Maps.  His tutorial used curated global data obtained from Kaggle to provide static and dynamic global maps.  This project used US time series data taken directly from the Github data repository for the 2019 Novel Coronavirus Visual Dashboard operated by the Johns Hopkins University Center for Systems Science and Engineering (JHU CSSE).  By using the original datasource, additional data preparation was required to ensure that the maps, particularly the dynamic map, displayed properly.  The original conception of this project would have created a dynamic map at the county level, but this seems to be beyond the capabilities of my current computer.

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import plotly as py
import plotly.express as px
import json
from urllib.request import urlopen

## Set Pandas Options

In [ ]:
pd.set_option('display.max_rows',15)

## Import Geographic Information

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

## Import US Covid-19 Case and Death Data

In [ ]:
cases_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
deaths_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'

In [ ]:
cases = pd.read_csv(cases_csv)
deaths = pd.read_csv(deaths_csv)

## Prepare Data for County Level Static Map

Limit data to counties that have at least one confirmed case/death

In [ ]:
today = '4/12/20'
cases = cases[cases[today] > 0]
deaths = deaths[deaths[today] > 0]

Add FIPS code for Dukes and Nantucket Counties in Massachuetts to just Dukes County (Martha's Vineyard).
Drop Kansas City, Missouri (data should be included already in counties)

In [ ]:
cases = cases.drop(labels=3148)
deaths = deaths.drop(labels=3148)
cases.at[3147,'FIPS'] = 25007
deaths.at[3147,'FIPS'] = 25007

Adjust FIPS codes

In [ ]:
cases['FIPS'] = cases['FIPS'].astype(int).astype(str).str.zfill(5)

In [ ]:
deaths['FIPS'] = deaths['FIPS'].astype(int).astype(str).str.zfill(5)

## Prepare Data for State Level Dynamic Map

Aggregate Data by State

In [ ]:
state_cases = cases.groupby(cases['Province_State']).sum()
state_deaths = deaths.groupby(deaths['Province_State']).sum()

case_dropped_rows = ['Diamond Princess', 'Grand Princess', 'Guam', 'Virgin Islands', 'Northern Mariana Islands']
death_dropped_rows = ['Guam','Northern Mariana Islands','Virgin Islands']
state_cases.drop(case_dropped_rows,inplace=True)
state_deaths.drop(death_dropped_rows, inplace=True)

state_deaths = state_deaths.reset_index()
state_cases = state_cases.reset_index()

Restructure DataFrame to work with Plotly animation

In [ ]:
def restructure(df):
    '''
    df:      Case or Death dataframe with aggregated state data
    returns: DataFrame restructured for Plotly animation
    '''
    states = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL',
              'GA','HI','ID','IL','IN','IA','KS','KY','LA','ME',
              'MD','MA','MI','MN','MS','MO','MT','NE','NV','NH',
              'NJ','NM','NY','NC','ND','OH','OK','OR','PA','PR',
              'RI','SC','SD','TN','TX','UT','VT','VA','WA','WV',
              'WI','WY']
    if len(df.index) == 51:
        states.pop()
    dates = list(df.columns.values)[11:]
    date_list = []
    for date in dates:
        animate = df[date].to_numpy()
        temp_array = np.array(list(zip(np.array(states), animate)))
        temp_df = pd.DataFrame(temp_array)
        temp_df['Date'] = date
        date_list.append(temp_df)
    return pd.concat(date_list)

animate_cases = restructure(state_cases).rename(columns={0: 'State', 1: 'Cases'})
animate_deaths = restructure(state_deaths).rename(columns={0: 'State', 1: 'Deaths'})

animate_cases['Cases'] = animate_cases['Cases'].astype(int)
animate_cases['Date'] = pd.to_datetime(animate_cases['Date']).astype(str)
animate_cases = animate_cases.sort_values(['State','Date'])

animate_deaths['Deaths'] = animate_deaths['Deaths'].astype(float).astype(int)
animate_deaths['Date'] = pd.to_datetime(animate_deaths['Date']).astype(str)
animate_deaths = animate_deaths.sort_values(['State','Date'])

## Create Static County Level Maps

In [ ]:
case_fig = px.choropleth(cases,
                         geojson=counties,
                         locations='FIPS',
                         color='4/7/20',
                         color_continuous_scale="Viridis",
                         range_color=(0, 50),
                         scope="usa",
                         labels={'4/7/20': 'cases as of 4/7/20'}
                        )

deaths_fig = px.choropleth(deaths,
                           geojson=counties,
                           locations='FIPS',
                           color='4/7/20',
                           color_continuous_scale="Viridis",
                           range_color=(0, 10),
                           scope="usa",
                           labels={'4/7/20': 'deaths as of 4/7/20'}
                          )

## Create Dynamic State Level Maps

In [ ]:
dynamic_cases_fig = px.choropleth(animate_cases,
                                  color= 'Cases',
                                  color_continuous_scale = "Inferno",
                                  range_color = (1,10000),
                                  locations = 'State',
                                  locationmode = 'USA-states',
                                  scope = 'usa',
                                  animation_frame = 'Date'
                                 )

dynamic_deaths_fig = px.choropleth(animate_deaths,
                                  color= 'Deaths',
                                  color_continuous_scale = "Inferno",
                                  range_color = (1,400),
                                  locations = 'State',
                                  locationmode = 'USA-states',
                                  scope = 'usa',
                                  animation_frame = 'Date'
                                 )

## Show Maps

In [ ]:
case_fig.show()
deaths_fig.show()
dynamic_cases_fig.show()
dynamic_deaths_fig.show()

In [ ]:
deaths

## Create 3-day percentage increase maps

Prepare data

In [ ]:
dates = list(deaths.columns.values)[12:]
cases['3 day percentage increase'] = (cases[dates[-2]] - cases[dates[-5]]) / cases[dates[-5]]
deaths['3 day percentage increase'] = (deaths[dates[-2]] - deaths[dates[-5]]) / deaths[dates[-5]]

Create map files

In [ ]:
cases_percentage_increase = px.choropleth(cases,
                                          geojson=counties,
                                          locations='FIPS',
                                          color = '3 day percentage increase',
                                          color_continuous_scale = "Viridis",
                                          range_color = (0,0.5),
                                          scope="usa"
                                         )

deaths_percentage_increase = px.choropleth(deaths,
                                           geojson=counties,
                                           locations='FIPS',
                                           color='3 day percentage increase',
                                           color_continuous_scale="Viridis",
                                           range_color = (0,0.5),
                                           scope="usa"
                                          )

Show maps

In [ ]:
cases_percentage_increase.show()
deaths_percentage_increase.show()

## Create Population Adjusted County Maps

Prepare data

In [ ]:
deaths['Per 100,000'] = deaths[today] * 100000 / deaths['Population']

Create Map File

In [ ]:
deaths_per_100000 = px.choropleth(deaths,
                                  geojson=counties,
                                  locations='FIPS',
                                  color='Per 100,000',
                                  color_continuous_scale="Viridis",
                                  range_color = (0,10),
                                  scope="usa"
                                 )

Show Map

In [ ]:
deaths_per_100000.show()